In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from pydataset import data
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


### Load the tips dataset.

- A. Create a column named tip_percentage. This should be the tip amount divided by the total bill.
- B. Create a column named price_per_person. This should be the total bill divided by the party size.
- C. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage?
- D. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?
- E. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?
- F. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

### A. Create a column named tip_percentage. This should be the tip amount divided by the total bill.

In [3]:
df['tip_percentage'] = df['tip']/ df['total_bill']
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


### B. Create a column named price_per_person. This should be the total bill divided by the party size. Need to rename the size variable as it is a reserved word in pandas.

In [4]:
df['price_per_person'] = df['total_bill'] / df['size']
df.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447,8.495000
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542,3.446667
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587,7.003333
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780,11.840000
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808,6.147500


### C. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? The tip percentage? 
### I would say the total_bill, day and time would be the important predictors of tip ammount

#### Maggies Code
from sklearn.feature_selection import SelectKBest, f_regression
f_selector = SelectKBest(f_regression, k=13)
f_selector = f_selector.fit(X_train_scaled, y_train.G3)
f_support = f_selector.get_support()
X_reduced_scaled = X_train_scaled.iloc[:,f_support]

from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
rfe = RFE(lm, 13)
X_rfe = rfe.fit_transform(X_train_scaled, y_train.G3)
mask = rfe.support_
X_reduced_scaled_rfe = X_train_scaled.iloc[:,mask]

### D. Use all the other numeric features to predict tip amount. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [5]:
train_validate, test = train_test_split(df, test_size=.2, 
                                        random_state=123)

train, validate = train_test_split(train_validate, 
                                   test_size=.3, random_state=123)

In [6]:
train.head()

,total_bill,tip,sex,smoker,day,time,size,tip_percentage,price_per_person
18,16.97,3.50,Female,No,Sun,Dinner,3,0.206246,5.656667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345,3.625000
118,12.43,1.80,Female,No,Thur,Lunch,2,0.144811,6.215000
28,21.70,4.30,Male,No,Sat,Dinner,2,0.198157,10.850000
237,32.83,1.17,Male,Yes,Sat,Dinner,2,0.035638,16.415000


In [7]:
train = train.drop(['sex','smoker','day','time'], axis = 1)
validate = validate.drop(['sex','smoker','day','time'], axis = 1)
test = test.drop(['sex','smoker','day','time'], axis = 1)
train.head()

,total_bill,tip,size,tip_percentage,price_per_person
18,16.97,3.50,3,0.206246,5.656667
172,7.25,5.15,2,0.710345,3.625000
118,12.43,1.80,2,0.144811,6.215000
28,21.70,4.30,2,0.198157,10.850000
237,32.83,1.17,2,0.035638,16.415000


In [8]:
# Going to set up all variables except tip
X_train = train.drop(columns=['tip'])
X_validate = validate.drop(columns=['tip'])
X_test = test.drop(columns=['tip'])

# Setting up just tip as my target variable
y_train = train[['tip']]
y_validate = validate[['tip']]
y_test = test[['tip']]

X_train.head()

,total_bill,size,tip_percentage,price_per_person
18,16.97,3,0.206246,5.656667
172,7.25,2,0.710345,3.625000
118,12.43,2,0.144811,6.215000
28,21.70,2,0.198157,10.850000
237,32.83,2,0.035638,16.415000


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(copy=True).fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [10]:
X_train_scaled = pd.DataFrame(X_train_scaled, 
                              columns=X_train.columns.values).\
                            set_index([X_train.index.values])

X_validate_scaled = pd.DataFrame(X_validate_scaled, 
                                columns=X_validate.columns.values).\
                            set_index([X_validate.index.values])

X_test_scaled = pd.DataFrame(X_test_scaled, 
                                columns=X_test.columns.values).\
                            set_index([X_test.index.values])
X_train_scaled.head()

,total_bill,size,tip_percentage,price_per_person
18,0.307114,0.4,0.252863,0.150344
172,0.092355,0.2,1.000000,0.032258
118,0.206805,0.2,0.161808,0.182796
28,0.411622,0.2,0.240873,0.452194
237,0.657534,0.2,0.000000,0.775647


In [11]:
# Numeric features include total_bill, tip, size, tip% and price_per_person
from sklearn.feature_selection import SelectKBest, f_regression

# Setting up the regression test to pick top 2 features to influence tip
f_selector = SelectKBest(f_regression, k=2)

# Setting up features to predict the target variable tip
X_train_scaled = X_train_scaled[['total_bill', 'size', 'tip_percentage', 'price_per_person']]
y_train = y_train[['tip']]

# fit the selector to data
f_selector.fit(X_train_scaled, y_train)

# storing array of boolean values that show true if a feature was selected as top 2
# false if not top 2
f_support = f_selector.get_support()
print(f_support) 

# Creating a list of the top 2 features using the array given above
f_feature = X_train_scaled.loc[:,f_support].columns.tolist()

print(str(len(f_feature)), 'selected features')
print(f_feature)


[ True  True False False]
2 selected features
['total_bill', 'size']


### Recursive Feature Elimination

In [12]:
# Initialize the linear regression object
lm = LinearRegression()

In [13]:
# Initialize the RFE object, setting the hyperparameters to be our 
# linear regression object above and the number of features to be returned
rfe = RFE(lm,2)

In [14]:
# Fit the RFE to our data. It creates multiple models and finds the best
# performing model. 
X_rfe = rfe.fit_transform(X_train_scaled,y_train)  

In [15]:
# Fit X_rfe for modeling
lm.fit(X_rfe, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [16]:
# If we want a list of the features that remain, we can use
# .support_ similar to how we used .get_support() with SelectKBest.
mask = rfe.support_

In [17]:
rfe_features = X_train_scaled.loc[:,mask].columns.tolist()

In [18]:
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['total_bill', 'tip_percentage']


### E. Use all the other numeric features to predict tip percentage. Use select k best and recursive feature elimination to select the top 2 features. What are they?

In [19]:
train.head()

,total_bill,tip,size,tip_percentage,price_per_person
18,16.97,3.50,3,0.206246,5.656667
172,7.25,5.15,2,0.710345,3.625000
118,12.43,1.80,2,0.144811,6.215000
28,21.70,4.30,2,0.198157,10.850000
237,32.83,1.17,2,0.035638,16.415000


In [20]:
# Going to set up all variables except tip
X_train2 = train.drop(columns=['tip_percentage'])
X_validate2 = validate.drop(columns=['tip_percentage'])
X_test2 = test.drop(columns=['tip_percentage'])

# Setting up just tip as my target variable
y_train2 = train[['tip_percentage']]
y_validate2 = validate[['tip_percentage']]
y_test2 = test[['tip_percentage']]

In [21]:
# Numeric features include total_bill, tip, size, tip% and price_per_person
from sklearn.feature_selection import SelectKBest, f_regression

# Setting up the regression test to pick top 2 features to influence tip
f_selector2 = SelectKBest(f_regression, k=2)

# Setting up features to predict the target variable tip_percentage
X_train2 = X_train2[['total_bill', 'size', 'tip', 'price_per_person']]
y_train2 = y_train2[['tip_percentage']]

# fit the selector to data
f_selector2.fit(X_train2, y_train2)

# storing array of boolean values that show true if a feature was selected as top 2
# false if not top 2
f_support2 = f_selector2.get_support()
print(f_support2) 

# Creating a list of the top 2 features using the array given above
f_feature2 = X_train2.loc[:,f_support2].columns.tolist()

print(str(len(f_feature2)), 'selected features')
print(f_feature2)

[False False  True  True]
2 selected features
['tip', 'price_per_person']


### Recursive Feature Elimination

In [22]:
# Initialize the linear regression object
lm2 = LinearRegression()

In [23]:
# Initialize the RFE object, setting the hyperparameters to be our 
# linear regression object above and the number of features to be returned
rfe2 = RFE(lm2,2)

In [24]:
# Fit the RFE to our data. It creates multiple models and finds the best
# performing model. 
X_rfe2 = rfe2.fit_transform(X_train2,y_train2)  

In [25]:
# Fit X_rfe for modeling
lm2.fit(X_rfe2, y_train2)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [26]:
# If we want a list of the features that remain, we can use
# .support_ similar to how we used .get_support() with SelectKBest.
mask2 = rfe2.support_

In [27]:
rfe_features = X_train2.loc[:,mask2].columns.tolist()

In [28]:
print(str(len(rfe_features)), 'selected features')
print(rfe_features)

2 selected features
['total_bill', 'tip']


### F. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

### 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [29]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_regression

def select_kbest(X_train, y_train, kn):
   
    # Setting up the regression test to pick top 2 features to influence tip
    f_selector = SelectKBest(f_regression, k=kn)

    # fit the selector to data
    f_selector.fit(X_train, y_train)

    # storing array of boolean values that show true if a feature was selected as top 2
    # false if not top 2
    f_support = f_selector.get_support()

    # Creating a list of the top 2 features using the array given above
    f_feature = X_train.loc[:,f_support].columns.tolist()

    print(str(len(f_feature)), 'selected features')
    print(f_feature)

In [30]:
X_train = X_train[['total_bill', 'size', 'tip_percentage', 'price_per_person']]
y_train = y_train[['tip']]
#X_train.head()
select_kbest(X_train,y_train,2)

2 selected features
['total_bill', 'size']


### 3.Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [31]:
def rfe(X_train, y_train, kn):
    # Initialize the linear regression object
    lm = LinearRegression()
    # Initialize the RFE object, setting the hyperparameters to be our 
    # linear regression object above and the number of features to be returned
    rfe = RFE(lm,kn)
    # Fit the RFE to our data. It creates multiple models and finds the best
    # performing model. 
    X_rfe = rfe.fit_transform(X_train,y_train)  
    # Fit X_rfe for modeling
    lm.fit(X_rfe, y_train)
    # If we want a list of the features that remain, we can use
    # .support_ similar to how we used .get_support() with SelectKBest.
    mask = rfe.support_
    rfe_features = X_train.loc[:,mask].columns.tolist()
    print(str(len(rfe_features)), 'selected features')
    print(rfe_features)

In [32]:
X_train = X_train[['total_bill', 'size', 'tip_percentage', 'price_per_person']]
y_train = y_train[['tip']]
rfe(X_train, y_train, 2)

2 selected features
['size', 'tip_percentage']


### 4.Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [33]:
df2 = data('swiss')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47 entries, Courtelary to Rive Gauche
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Fertility         47 non-null     float64
 1   Agriculture       47 non-null     float64
 2   Examination       47 non-null     int64  
 3   Education         47 non-null     int64  
 4   Catholic          47 non-null     float64
 5   Infant.Mortality  47 non-null     float64
dtypes: float64(4), int64(2)
memory usage: 2.6+ KB


In [34]:
df2.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [35]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 18 to 166
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   total_bill        136 non-null    float64
 1   tip               136 non-null    float64
 2   size              136 non-null    int64  
 3   tip_percentage    136 non-null    float64
 4   price_per_person  136 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 6.4 KB


In [36]:
x = df2.drop(columns = 'Fertility')
y = df2['Fertility']

select_kbest(x,y,3)

3 selected features
['Examination', 'Education', 'Catholic']


In [37]:
x = df2.drop(columns = 'Fertility')
y = df2['Fertility']
rfe(x,y,3)

3 selected features
['Examination', 'Education', 'Infant.Mortality']


In [38]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   explore_telco.ipynb
	modified:   feature_engineering_walkthru.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	feature_engineering.ipynb
	student/
	zillow_regression_project/

no changes added to commit (use "git add" and/or "git commit -a")


In [39]:
! git add -A

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> zillow_regression_project
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached zillow_regression_project
hint: 
hint: See "git help submodule" for more information.


In [40]:
! git rm --cached zillow_regression_project

error: the following file has staged content different from both the
file and the HEAD:
    zillow_regression_project
(use -f to force removal)


In [43]:
! git rm -f --cached zillow_regression_project

rm 'zillow_regression_project'


In [44]:
! git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   explore_telco.ipynb
	new file:   feature_engineering.ipynb
	modified:   feature_engineering_walkthru.ipynb
	new file:   student/student-merge.R
	new file:   student/student.txt

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   feature_engineering.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	zillow_regression_project/



In [45]:
! git add - 

hint: You've added another git repository inside your current repository.
hint: Clones of the outer repository will not contain the contents of
hint: the embedded repository and will not know how to obtain it.
hint: If you meant to add a submodule, use:
hint: 
hint: 	git submodule add <url> zillow_regression_project
hint: 
hint: If you added this path by mistake, you can remove it from the
hint: index with:
hint: 
hint: 	git rm --cached zillow_regression_project
hint: 
hint: See "git help submodule" for more information.
